In [1]:
import warnings
warnings.simplefilter('ignore')

import os
import re
import gc
import random
import pickle

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [2]:
%%time

df = []
list_dir = os.listdir('./wiki_data')
list_dir = sorted(list_dir)
for path in list_dir:
    if 'wiki_2023_index' in path or 'deberta' in path or 'my' in path or 'ipynb' in path:
        continue
    print(path)
    x = pd.read_parquet(os.path.join('./wiki_data', path))
    x['file'] = path
    df.append(x)
df = pd.concat(df, axis=0).reset_index(drop=True)
df['id'] = df['id'].astype(int)

display(df)

a.parquet
b.parquet
c.parquet
d.parquet
e.parquet
f.parquet
g.parquet
h.parquet
i.parquet
j.parquet
k.parquet
l.parquet
m.parquet
n.parquet
number.parquet
o.parquet
other.parquet
p.parquet
q.parquet
r.parquet
s.parquet
t.parquet
u.parquet
v.parquet
w.parquet
x.parquet
y.parquet
z.parquet


,id,title,text,categories,file
0,49495844,A & B High Performance Firearms,A & B High Performance Firearms was a competit...,"[Defunct firearms manufacturers, Defunct manuf...",a.parquet
1,3579086,A & C Black,A & C Black is a British book publishing compa...,"[Encyclopædia Britannica, Ornithological publi...",a.parquet
2,62397582,A & F Harvey Brothers,"A & F Harvey Brothers, first Spinning Cotton M...",[Cotton mills],a.parquet
3,15547032,A & G Price,A & G Price Limited is an engineering firm and...,"[Locomotive manufacturers of New Zealand, Tham...",a.parquet
4,8021609,A & M Karagheusian,thumb|right|238px|A portion of the Karagheusia...,"[1904 establishments in the United States, Arm...",a.parquet
...,...,...,...,...,...
6286770,18881188,Zəyəm Cırdaxan,"Zəyəm Cırdaxan (also Dzegam-Dzhirdakhan, Dzega...",[Populated places in Shamkir District],z.parquet
6286771,18873112,"Zəyəm, Qakh","Zəyəm (also, Zagam, Zagyam, and Zeyam; ) is a ...",[Populated places in Qakh District],z.parquet
6286772,18881204,"Zəyəm, Shamkir","Zəyəm (also, Dzagam, Dzegam, Dzegan, Guseynbei...",[Populated places in Shamkir District],z.parquet
6286773,18920475,"Zəyəm, Zaqatala","Zəyəm (also, Zagam and Zagyam) is a village an...",[Populated places in Zaqatala District],z.parquet


CPU times: user 1min 8s, sys: 27.4 s, total: 1min 36s
Wall time: 2min 7s


In [5]:
val_ids = pd.read_parquet('./data/recall_val_Top1000.parquet', columns=['page_id'])
val_ids['page_id'] = val_ids['page_id'].astype(int)
val_ids = val_ids['page_id'].values.tolist()

import pickle as pkl
with open('./tmp/7w_Top1000.pkl','rb') as f:
    train = pkl.load(f)
# train = pd.read_parquet('./data/7w_Top1000.parquet')
train['page_id'] = train['page_id'].astype(int)
valid = train[train['page_id'].isin(val_ids)].copy().reset_index(drop=True)
train = train[~train['page_id'].isin(val_ids)].copy().reset_index(drop=True)

display(train)
display(valid)

,prompt,A,B,C,D,E,answer,page_id,page_title,Top1000
0,What does the nullity of a graph in graph theo...,The number of vertices in the graph,The number of edges in the graph,The rank of the adjacency matrix,The multiplicity of the eigenvalue 0 in the sp...,The number of components of the graph,D,17458663,Nullity (graph theory),"[3876952, 4778094, 74006, 3554666, 3876892, 38..."
1,"In the matrix theory of graphs, what is the nu...",n − r,m − n + c,n − c,The nullity of the graph,The number of components of the graph,A,17458663,Nullity (graph theory),"[3876952, 4778094, 74006, 3554666, 74004, 1993..."
2,Which term is more commonly used to refer to t...,Nullity of the adjacency matrix,Multiplicity of the eigenvalue 0,Cycle rank,Cyclomatic number,Circuit rank,C,17458663,Nullity (graph theory),"[3876952, 4778094, 1015508, 592872, 610861, 74..."
3,What is the formula for calculating the nullit...,n − r,m − n + c,n − c,The nullity of the graph,The number of components of the graph,B,17458663,Nullity (graph theory),"[3876952, 4778094, 1993693, 3418262, 3418253, ..."
4,What can the nullity of the graph represent in...,The number of vertices in the graph,The number of edges in the graph,The rank of the adjacency matrix,The number of components of the graph,The rank of the oriented incidence matrix,E,17458663,Nullity (graph theory),"[3876952, 4778094, 5317850, 3418253, 74006, 34..."
...,...,...,...,...,...,...,...,...,...,...
75306,What is the inscription on the plinth of the a...,Rotary Armillary,Service Above Self,This Armillary was presented to the people of ...,The Rotary Club of Cheam has joined with the R...,December 2000,B,43629419,Sutton armillary,"[5456608, 1050997, 1117151, 1051257, 1050718, ..."
75307,How was the Millennium Dial Armillary funded?,By the London Borough of Sutton,By the Rotary Club,By the sponsors listed on the plinth inscription,By the residents of Sutton,By a joint effort between the Rotary Club and ...,C,43629419,Sutton armillary,"[5456608, 5158176, 6080877, 1634041, 5605385, ..."
75308,What is the Ukrainian Physicists' Tournament f...,A competition for secondary school students to...,A competition for university students to solve...,A competition for international physicists to ...,A research project competition for university ...,A creative team competition for university stu...,E,14320529,Ukrainian Physicists' Tournament for Universit...,"[5873747, 2365219, 5894257, 2366735, 1709283, ..."
75309,When was the first Open Physicists' Tournament...,2001,2005,2010,2015,2020,A,14320529,Ukrainian Physicists' Tournament for Universit...,"[5873747, 5894257, 1709283, 2365219, 1618922, ..."


,prompt,A,B,C,D,E,answer,page_id,page_title,Top1000
0,What is the definition of a solvent?,A substance that can change itself chemically,A substance that can dilute or extract other s...,A substance that can change other substances c...,A substance that can only dissolve other subst...,A substance that can only dilute other substan...,B,73305281,Green solvent,"[5285194, 5897602, 2004006, 5285209, 3596063, ..."
1,What are some adverse effects associated with ...,Burns and eczema,Cancer and brain disorder,Fetotoxicity and inflammations of peripheral n...,All of the above,None of the above,D,73305281,Green solvent,"[2004006, 4540298, 5285194, 5285209, 5285211, ..."
2,What is the goal of the UN sustainability deve...,To promote the use of petrochemical solvents,To develop green and sustainable solvents,To increase the use of nonpolar solvents,To classify solvents into polar and nonpolar c...,To decrease the use of solvents in various dom...,B,73305281,Green solvent,"[2004006, 4540298, 4540350, 5285209, 4540112, ..."
3,What are green solvents derived from?,Processing of agricultural crops,Petrochemical sources,Regular solvents,Chemical reactions,Water and air pollution,A,73305281,Green solvent,"[2004006, 5285209, 5285194, 5285200, 5233230, ..."
4,What makes it difficult to classify green solv...,Their chemical structure and source can differ,They have adverse effects on health and safety,They are associated with water and air pollution,They are derived from petrochemical sources,They are nonpolar solvents,A,73305281,Green solvent,"[2004006, 5285209, 5233230, 4540298, 5285200, ..."
...,...,...,...,...,...,...,...,...,...,...
2692,Who is the Claud A. Jones Award named after?,Admiral Claud Ashton Jones,Commander Carl N.,Medal of Honor recipient Admiral Claud Ashton ...,Medal of Honor recipient Commander Carl N.,Commander Claud Ashton Jones,A,28010431,Claud A. Jones Award,"[1029280, 1029285, 255533, 6136400, 75191, 257..."
2693,What are the criteria for nomination for the C...,Making significant contributions to improving ...,Having served in the United States fleet or fi...,Being endorsed by a select number of experienc...,Having a career spanning over 40 years includi...,Being named after a Medal of Honor recipient,A,28010431,Claud A. Jones Award,"[1029280, 1029285, 255533, 2132169, 2457379, 1..."
2694,When was the Komatsu 960E-1 introduced?,"May 27, 2008",September 2016,"Peoria, Illinois",360 short tons,Komatsu North America,A,26398206,Komatsu 960E-1,"[2843449, 2843448, 2419712, 782981, 2843447, 3..."
2695,What is the power output of the diesel engine ...,"3,500 horsepower","2,600 kW",Komatsu SSDA18V170 V18,General Electric GTA-39,Industrial Power Alliance,A,26398206,Komatsu 960E-1,"[2843449, 3442486, 2843447, 2843448, 5458389, ..."


In [6]:
index2id = df.to_dict()['id']

In [7]:
def func(x):
    return [index2id[i] for i in x]

train['Top1000'] = train['Top1000'].apply(func)
valid['Top1000'] = valid['Top1000'].apply(func)

display(train)
display(valid)

,prompt,A,B,C,D,E,answer,page_id,page_title,Top1000
0,What does the nullity of a graph in graph theo...,The number of vertices in the graph,The number of edges in the graph,The rank of the adjacency matrix,The multiplicity of the eigenvalue 0 in the sp...,The number of components of the graph,D,17458663,Nullity (graph theory),"[17458663, 17458495, 244463, 39378034, 1520758..."
1,"In the matrix theory of graphs, what is the nu...",n − r,m − n + c,n − c,The nullity of the graph,The number of components of the graph,A,17458663,Nullity (graph theory),"[17458663, 17458495, 244463, 39378034, 2535016..."
2,Which term is more commonly used to refer to t...,Nullity of the adjacency matrix,Multiplicity of the eigenvalue 0,Cycle rank,Cyclomatic number,Circuit rank,C,17458663,Nullity (graph theory),"[17458663, 17458495, 1483049, 13511620, 368481..."
3,What is the formula for calculating the nullit...,n − r,m − n + c,n − c,The nullity of the graph,The number of components of the graph,B,17458663,Nullity (graph theory),"[17458663, 17458495, 745714, 37123010, 244321,..."
4,What can the nullity of the graph represent in...,The number of vertices in the graph,The number of edges in the graph,The rank of the adjacency matrix,The number of components of the graph,The rank of the oriented incidence matrix,E,17458663,Nullity (graph theory),"[17458663, 17458495, 66468324, 244321, 244463,..."
...,...,...,...,...,...,...,...,...,...,...
75306,What is the inscription on the plinth of the a...,Rotary Armillary,Service Above Self,This Armillary was presented to the people of ...,The Rotary Club of Cheam has joined with the R...,December 2000,B,43629419,Sutton armillary,"[43629419, 18608917, 33943893, 41372128, 77297..."
75307,How was the Millennium Dial Armillary funded?,By the London Borough of Sutton,By the Rotary Club,By the sponsors listed on the plinth inscription,By the residents of Sutton,By a joint effort between the Rotary Club and ...,C,43629419,Sutton armillary,"[43629419, 1306104, 7486135, 39644286, 1069951..."
75308,What is the Ukrainian Physicists' Tournament f...,A competition for secondary school students to...,A competition for university students to solve...,A competition for international physicists to ...,A research project competition for university ...,A creative team competition for university stu...,E,14320529,Ukrainian Physicists' Tournament for Universit...,"[14320529, 42727309, 36532984, 7208965, 567743..."
75309,When was the first Open Physicists' Tournament...,2001,2005,2010,2015,2020,A,14320529,Ukrainian Physicists' Tournament for Universit...,"[14320529, 36532984, 5677430, 42727309, 332583..."


,prompt,A,B,C,D,E,answer,page_id,page_title,Top1000
0,What is the definition of a solvent?,A substance that can change itself chemically,A substance that can dilute or extract other s...,A substance that can change other substances c...,A substance that can only dissolve other subst...,A substance that can only dilute other substan...,B,73305281,Green solvent,"[37431, 2740944, 73305281, 5048079, 229643, 13..."
1,What are some adverse effects associated with ...,Burns and eczema,Cancer and brain disorder,Fetotoxicity and inflammations of peripheral n...,All of the above,None of the above,D,73305281,Green solvent,"[73305281, 47760832, 37431, 5048079, 3708060, ..."
2,What is the goal of the UN sustainability deve...,To promote the use of petrochemical solvents,To develop green and sustainable solvents,To increase the use of nonpolar solvents,To classify solvents into polar and nonpolar c...,To decrease the use of solvents in various dom...,B,73305281,Green solvent,"[73305281, 47760832, 37161565, 5048079, 67728,..."
3,What are green solvents derived from?,Processing of agricultural crops,Petrochemical sources,Regular solvents,Chemical reactions,Water and air pollution,A,73305281,Green solvent,"[73305281, 5048079, 37431, 3528528, 20064295, ..."
4,What makes it difficult to classify green solv...,Their chemical structure and source can differ,They have adverse effects on health and safety,They are associated with water and air pollution,They are derived from petrochemical sources,They are nonpolar solvents,A,73305281,Green solvent,"[73305281, 5048079, 20064295, 47760832, 352852..."
...,...,...,...,...,...,...,...,...,...,...
2692,Who is the Claud A. Jones Award named after?,Admiral Claud Ashton Jones,Commander Carl N.,Medal of Honor recipient Admiral Claud Ashton ...,Medal of Honor recipient Commander Carl N.,Commander Claud Ashton Jones,A,28010431,Claud A. Jones Award,"[28010431, 1610873, 37155106, 37090957, 432511..."
2693,What are the criteria for nomination for the C...,Making significant contributions to improving ...,Having served in the United States fleet or fi...,Being endorsed by a select number of experienc...,Having a career spanning over 40 years includi...,Being named after a Medal of Honor recipient,A,28010431,Claud A. Jones Award,"[28010431, 1610873, 37155106, 70787728, 16544,..."
2694,When was the Komatsu 960E-1 introduced?,"May 27, 2008",September 2016,"Peoria, Illinois",360 short tons,Komatsu North America,A,26398206,Komatsu 960E-1,"[26398206, 12074089, 38673715, 35058939, 13094..."
2695,What is the power output of the diesel engine ...,"3,500 horsepower","2,600 kW",Komatsu SSDA18V170 V18,General Electric GTA-39,Industrial Power Alliance,A,26398206,Komatsu 960E-1,"[26398206, 1279370, 13094835, 12074089, 652067..."


In [8]:
page_ids = list(df['id'].unique())

train_filted = train[train['page_id'].isin(page_ids)].reset_index(drop=True)
valid_filted = valid[valid['page_id'].isin(page_ids)].reset_index(drop=True)

display(train_filted)
display(valid_filted)

,prompt,A,B,C,D,E,answer,page_id,page_title,Top1000
0,What does the nullity of a graph in graph theo...,The number of vertices in the graph,The number of edges in the graph,The rank of the adjacency matrix,The multiplicity of the eigenvalue 0 in the sp...,The number of components of the graph,D,17458663,Nullity (graph theory),"[17458663, 17458495, 244463, 39378034, 1520758..."
1,"In the matrix theory of graphs, what is the nu...",n − r,m − n + c,n − c,The nullity of the graph,The number of components of the graph,A,17458663,Nullity (graph theory),"[17458663, 17458495, 244463, 39378034, 2535016..."
2,Which term is more commonly used to refer to t...,Nullity of the adjacency matrix,Multiplicity of the eigenvalue 0,Cycle rank,Cyclomatic number,Circuit rank,C,17458663,Nullity (graph theory),"[17458663, 17458495, 1483049, 13511620, 368481..."
3,What is the formula for calculating the nullit...,n − r,m − n + c,n − c,The nullity of the graph,The number of components of the graph,B,17458663,Nullity (graph theory),"[17458663, 17458495, 745714, 37123010, 244321,..."
4,What can the nullity of the graph represent in...,The number of vertices in the graph,The number of edges in the graph,The rank of the adjacency matrix,The number of components of the graph,The rank of the oriented incidence matrix,E,17458663,Nullity (graph theory),"[17458663, 17458495, 66468324, 244321, 244463,..."
...,...,...,...,...,...,...,...,...,...,...
68889,What is the inscription on the plinth of the a...,Rotary Armillary,Service Above Self,This Armillary was presented to the people of ...,The Rotary Club of Cheam has joined with the R...,December 2000,B,43629419,Sutton armillary,"[43629419, 18608917, 33943893, 41372128, 77297..."
68890,How was the Millennium Dial Armillary funded?,By the London Borough of Sutton,By the Rotary Club,By the sponsors listed on the plinth inscription,By the residents of Sutton,By a joint effort between the Rotary Club and ...,C,43629419,Sutton armillary,"[43629419, 1306104, 7486135, 39644286, 1069951..."
68891,What is the Ukrainian Physicists' Tournament f...,A competition for secondary school students to...,A competition for university students to solve...,A competition for international physicists to ...,A research project competition for university ...,A creative team competition for university stu...,E,14320529,Ukrainian Physicists' Tournament for Universit...,"[14320529, 42727309, 36532984, 7208965, 567743..."
68892,When was the first Open Physicists' Tournament...,2001,2005,2010,2015,2020,A,14320529,Ukrainian Physicists' Tournament for Universit...,"[14320529, 36532984, 5677430, 42727309, 332583..."


,prompt,A,B,C,D,E,answer,page_id,page_title,Top1000
0,What is the definition of a solvent?,A substance that can change itself chemically,A substance that can dilute or extract other s...,A substance that can change other substances c...,A substance that can only dissolve other subst...,A substance that can only dilute other substan...,B,73305281,Green solvent,"[37431, 2740944, 73305281, 5048079, 229643, 13..."
1,What are some adverse effects associated with ...,Burns and eczema,Cancer and brain disorder,Fetotoxicity and inflammations of peripheral n...,All of the above,None of the above,D,73305281,Green solvent,"[73305281, 47760832, 37431, 5048079, 3708060, ..."
2,What is the goal of the UN sustainability deve...,To promote the use of petrochemical solvents,To develop green and sustainable solvents,To increase the use of nonpolar solvents,To classify solvents into polar and nonpolar c...,To decrease the use of solvents in various dom...,B,73305281,Green solvent,"[73305281, 47760832, 37161565, 5048079, 67728,..."
3,What are green solvents derived from?,Processing of agricultural crops,Petrochemical sources,Regular solvents,Chemical reactions,Water and air pollution,A,73305281,Green solvent,"[73305281, 5048079, 37431, 3528528, 20064295, ..."
4,What makes it difficult to classify green solv...,Their chemical structure and source can differ,They have adverse effects on health and safety,They are associated with water and air pollution,They are derived from petrochemical sources,They are nonpolar solvents,A,73305281,Green solvent,"[73305281, 5048079, 20064295, 47760832, 352852..."
...,...,...,...,...,...,...,...,...,...,...
2522,Who is the Claud A. Jones Award named after?,Admiral Claud Ashton Jones,Commander Carl N.,Medal of Honor recipient Admiral Claud Ashton ...,Medal of Honor recipient Commander Carl N.,Commander Claud Ashton Jones,A,28010431,Claud A. Jones Award,"[28010431, 1610873, 37155106, 37090957, 432511..."
2523,What are the criteria for nomination for the C...,Making significant contributions to improving ...,Having served in the United States fleet or fi...,Being endorsed by a select number of experienc...,Having a career spanning over 40 years includi...,Being named after a Medal of Honor recipient,A,28010431,Claud A. Jones Award,"[28010431, 1610873, 37155106, 70787728, 16544,..."
2524,When was the Komatsu 960E-1 introduced?,"May 27, 2008",September 2016,"Peoria, Illinois",360 short tons,Komatsu North America,A,26398206,Komatsu 960E-1,"[26398206, 12074089, 38673715, 35058939, 13094..."
2525,What is the power output of the diesel engine ...,"3,500 horsepower","2,600 kW",Komatsu SSDA18V170 V18,General Electric GTA-39,Industrial Power Alliance,A,26398206,Komatsu 960E-1,"[26398206, 1279370, 13094835, 12074089, 652067..."


In [9]:
cnt = 0
for _, row in tqdm(train_filted.iterrows(), total=len(train_filted)):
    if row['page_id'] in row['Top1000']:
        cnt += 1
print(cnt, cnt/len(train_filted))

100%|██████████| 68894/68894 [00:02<00:00, 23053.74it/s]

68865 0.999579063488838


In [10]:
df.rename(columns={'id': 'candidate'}, inplace=True)
df.drop(['file', 'categories'], axis=1, inplace=True)

In [11]:
%%time

train_filted['prompt_answer'] = train_filted.apply(
    lambda row : ' '.join(str(row[x]) for x in ['prompt', 'A', 'B', 'C', 'D', 'E']),axis=1)
train = train_filted[['prompt_answer', 'page_id', 'Top1000']].copy()
train['id'] = train.index
train = train.explode('Top1000').reset_index(drop=True)
train.rename(columns={'Top1000': 'candidate', 'page_id': 'target'}, inplace=True)
train = pd.merge(train, df, on='candidate', how='left')
train['target'] = (train['target'] == train['candidate']).astype(int)
train['recall_rank'] = train.groupby('prompt_answer').cumcount()+1
train

CPU times: user 1min 24s, sys: 9.39 s, total: 1min 33s
Wall time: 1min 33s


,prompt_answer,target,candidate,id,title,text,recall_rank
0,What does the nullity of a graph in graph theo...,1,17458663,0,Nullity (graph theory),The nullity of a graph in the mathematical sub...,1
1,What does the nullity of a graph in graph theo...,0,17458495,0,Rank (graph theory),"In graph theory, a branch of mathematics, the ...",2
2,What does the nullity of a graph in graph theo...,0,244463,0,Adjacency matrix,"In graph theory and computer science, an adjac...",3
3,What does the nullity of a graph in graph theo...,0,39378034,0,Minimum rank of a graph,"In mathematics, the minimum rank is a graph pa...",4
4,What does the nullity of a graph in graph theo...,0,1520758,0,Null graph,"In the mathematical field of graph theory, the...",5
...,...,...,...,...,...,...,...
68893995,What is the International Physicists' Tourname...,0,61374991,68893,2007 Men's EuroHockey Nations Challenge I,The 2007 Men's EuroHockey Nations Challenge I ...,996
68893996,What is the International Physicists' Tourname...,0,12094774,68893,World Junior Teams Championship,The World Junior Teams Championship is a bridg...,997
68893997,What is the International Physicists' Tourname...,0,66467705,68893,2020–21 ISU Speed Skating World Cup – World Cup 2,The second and last competition weekend of the...,998
68893998,What is the International Physicists' Tourname...,0,65740427,68893,Dmitriy Gorbunov,DmitriyOn Dartsdatabase and in some articles o...,999


In [12]:
%%time

valid_filted['prompt_answer'] = valid_filted.apply(
    lambda row : ' '.join(str(row[x]) for x in ['prompt', 'A', 'B', 'C', 'D', 'E']),axis=1)
valid = valid_filted[['prompt_answer', 'page_id', 'Top1000']].copy()
valid['id'] = valid.index
valid = valid.explode('Top1000').reset_index(drop=True)
valid.rename(columns={'Top1000': 'candidate', 'page_id': 'target'}, inplace=True)
valid = pd.merge(valid, df, on='candidate', how='left')
valid['target'] = (valid['target'] == valid['candidate']).astype(int)
valid['recall_rank'] = valid.groupby('prompt_answer').cumcount()+1
valid

CPU times: user 7.26 s, sys: 1.58 s, total: 8.84 s
Wall time: 8.84 s


,prompt_answer,target,candidate,id,title,text,recall_rank
0,What is the definition of a solvent? A substan...,0,37431,0,Solvent,"thumb|Ethyl acetate, nail polish solvent. A so...",1
1,What is the definition of a solvent? A substan...,0,2740944,0,Universal solvent,"Universal solvent may refer to: *Water, descri...",2
2,What is the definition of a solvent? A substan...,1,73305281,0,Green solvent,A solvent is a substance that is liquid at the...,3
3,What is the definition of a solvent? A substan...,0,5048079,0,Solvent dye,A solvent dye is a dye soluble in organic solv...,4
4,What is the definition of a solvent? A substan...,0,229643,0,Molality,Molality is a measure of the number of moles o...,5
...,...,...,...,...,...,...,...
2526995,"Where is the Komatsu 960E-1 assembled? Peoria,...",0,25121593,2526,KOA Corporation,"KOA Corporation (Japanese: コーア株式会社, Kōa kabush...",996
2526996,"Where is the Komatsu 960E-1 assembled? Peoria,...",0,15040535,2526,Mitsubishi Colt 1000,The Mitsubishi Colt (A20) was one of their fir...,997
2526997,"Where is the Komatsu 960E-1 assembled? Peoria,...",0,62697879,2526,KTM 250 FRR,The KTM 250 FRR was a racing motorcycle made b...,998
2526998,"Where is the Komatsu 960E-1 assembled? Peoria,...",0,1054261,2526,Mitsubishi 6A1 engine,The Mitsubishi 6A1 engine is a series of pisto...,999


In [13]:
# 数据太多了, train 按 id 采样

sample_ids = random.sample(train['id'].unique().tolist(), k=30000)
train = train[train['id'].isin(sample_ids)].sort_values(
    ['id', 'recall_rank'], ascending=[True, True]).reset_index(drop=True)
display(train)

,prompt_answer,target,candidate,id,title,text,recall_rank
0,What is the formula for calculating the nullit...,1,17458663,3,Nullity (graph theory),The nullity of a graph in the mathematical sub...,1
1,What is the formula for calculating the nullit...,0,17458495,3,Rank (graph theory),"In graph theory, a branch of mathematics, the ...",2
2,What is the formula for calculating the nullit...,0,745714,3,Graphic matroid,"In the mathematical theory of matroids, a grap...",3
3,What is the formula for calculating the nullit...,0,37123010,3,Matroid rank,"In the mathematical theory of matroids, the ra...",4
4,What is the formula for calculating the nullit...,0,244321,3,Matroid,"In combinatorics, a branch of mathematics, a m...",5
...,...,...,...,...,...,...,...
29999995,What is the Ukrainian Physicists' Tournament f...,0,68001314,68891,Andrey Andreychenko,Andrey Valeryevich Andreychenko (; born 29 Mar...,996
29999996,What is the Ukrainian Physicists' Tournament f...,0,52096152,68891,2016 European championships international drau...,The 2016 European championships of internation...,997
29999997,What is the Ukrainian Physicists' Tournament f...,0,53113196,68891,Curling at the 2017 Winter Universiade,Curling at the 2017 Winter Universiade was hel...,998
29999998,What is the Ukrainian Physicists' Tournament f...,0,57749759,68891,2018–19 Ukrainian Football Amateur League,The 2018–19 Ukrainian Football Amateur League ...,999


In [14]:
%%time

train.to_pickle('./data/train_data.pkl')
valid.to_pickle('./data/valid_data.pkl')

CPU times: user 38.5 s, sys: 15.5 s, total: 54 s
Wall time: 54.7 s
